In [68]:
from abc import ABCMeta, abstractmethod

class Commutative_Ring(metaclass = ABCMeta):              #定义抽象基类表示交换环
    @abstractmethod
    def __add__(self, other):                                 #定义子类必须要重载的运算符 + 和 *           
        pass
    
    @abstractmethod
    def __mul__(self, other):
        pass

In [69]:
class Complex_Number(Commutative_Ring):                    #定义复数, 为交换环的子类
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Complex_Number(other.x + self.x, other.y + self.y)
    
    def __mul__(self, other):
        return Complex_Number(other.x * self.x - other.y * self.y, other.x * self.y + other.y * self.x)
    
    def __str__(self):                                  #用复平面下的坐标来表示复数
        return "(" + str(self.x) + ", " + str(self.y) + ")"

#测试
x1 = Complex_Number(1, 1)
x2 = Complex_Number(1, 2) 
print("x1 * x1 = %s"  %(x1 * x1))                                   #(1 + i) * (1 + i) = 2i               
print("x1 + x2 = %s, x2 + x1 = %s" %(x1 + x2, x2 + x1))            #(1 + i) + (1 + 2i) = 2 + 3i
print("x1 * x2 = %s, x2 * x1 = %s" %(x1 * x2, x2 * x1))            #(1 + i) * (1 + 2i) = -1 + 3i

x1 * x1 = (0, 2)
x1 + x2 = (2, 3), x2 + x1 = (2, 3)
x1 * x2 = (-1, 3), x2 * x1 = (-1, 3)


In [70]:
class Real_Cofficient_Poly(Commutative_Ring):              #定义实系数多项式环, 为交换环的子类
    def __init__(self, L):                                 #用一个列表记录多项式的系数，若是加法恒元，则为[]， 否则长度为deg + 1
        self.L = L[:]
    
    def __add__(self, other):                               #重载加法
        L1 = []
        if(len(self.L) < len(other.L)):
            for i in range (len(self.L)):
                L1.append(self.L[i] + other.L[i])
            L1 = L1 + other.L[len(self.L):]
        else:
            for i in range (len(other.L)):
                L1.append(self.L[i] + other.L[i])
            L1 = L1 + self.L[len(other.L):]
        length = len(L1)
        
        while(length > 0 and L1[length-1] == 0):               #判断最高项系数是否为0
            L1.pop()
            length -= 1
        return Real_Cofficient_Poly(L1)
    
    def __neg__(self):                                      #定义加法逆元的表示
        L1 = self.L[:]
        for i in range(len(L1)):
            L1[i] = -L1[i]
        return Real_Cofficient_Poly(L1)
    
    def __mul__(self, other):
        if(len(self.L) == 0 or len(other.L) == 0):                #若是有一项是加法恒元
            return Real_Cofficient_Poly([])
        
        L1 = [0 for i in range(len(self.L)+len(other.L) - 1)] #若是两个多项式都有次数的定义，则现在的degree = self.L.degree + other.L.degree
        for i in range(len(self.L)):
            for j in range(len(other.L)):
                L1[i+j] += self.L[i] * other.L[j]
        return Real_Cofficient_Poly(L1)
    
    def __str__(self):                                              #只表示出系数
        if(len(self.L) == 0):
            return "(0, 0, 0...)"
        else:
            s = "(" 
            for i in range(len(self.L)):
                s += str(self.L[i]) + ", "
            s += "0...)"
        return s
            
    
#测试
y1 = Real_Cofficient_Poly([1, 1])
y2 = Real_Cofficient_Poly([3, 2, 1])
print("y1 = %s, y2 = %s" %(y1, y2))
print("y1 * y1 * y1 = %s"  %(y1 * y1 * y1))                      #(x +1)^3 = x^3 + 3x^2 + 3x + 1
print("y1的加法逆元为  %s" %(-y1))
print("y1 + y2 = %s, y1 * y2 = %s" %(y1 + y2, y1 * y2))              #(x +1)(x^2 + 2x + 3)= x^3 + 3x^2 + 5x + 3

y1 = (1, 1, 0...), y2 = (3, 2, 1, 0...)
y1 * y1 * y1 = (1, 3, 3, 1, 0...)
y1的加法逆元为  (-1, -1, 0...)
y1 + y2 = (4, 3, 1, 0...), y1 * y2 = (3, 5, 3, 1, 0...)


In [71]:
class Frac_RealCoff_Poly(Commutative_Ring):                # 定义实系数多项式环的分式域, 为交换环的子类
    def __init__(self, poly1, poly2):                                 #分式域的每个元素由两个多项式环元素构成
        self.poly1 = Real_Cofficient_Poly(poly1.L)
        self.poly2 = Real_Cofficient_Poly(poly2.L)
    
    def __add__(self, other):                               #重载加法, [a, b] + [c, d] = [ad + bc, bd]
        return Frac_RealCoff_Poly(self.poly1 * other.poly2 + self.poly2 * other.poly1, self.poly2 * other.poly2)
    
    def __mul__(self, other):                                #重载乘法, [a, b] * [c, d] = [ac, bd]
        return Frac_RealCoff_Poly(self.poly1 * other.poly1, self.poly2 * other.poly2)
    
    def __str__(self):
        return "[" + str(self.poly1) + ", " + str(self.poly2) + "]"

#测试
z1 = Frac_RealCoff_Poly(y1, y2)
z2 = Frac_RealCoff_Poly(Real_Cofficient_Poly([3]), Real_Cofficient_Poly([0, 1]))
print("z1 = %s, z2 = %s" %(z1, z2))
print("z1 + z2 = %s" %(z1 + z2))
print("z1 * z2 = %s" %(z1 * z2))

z1 = [(1, 1, 0...), (3, 2, 1, 0...)], z2 = [(3, 0...), (0, 1, 0...)]
z1 + z2 = [(9, 7, 4, 0...), (0, 3, 2, 1, 0...)]
z1 * z2 = [(3, 3, 0...), (0, 3, 2, 1, 0...)]
